In [ ]:
import requests
import cv2
import numpy as np
import time
import threading
from urllib.parse import urlparse

# Ganti dengan IP ESP32-CAM Anda
ESP32_IP = "192.168.1.167"

def test_basic_connection():
    """Test koneksi dasar ke ESP32-CAM"""
    print("Testing basic connection...")
    try:
        response = requests.get(f"http://{ESP32_IP}/", timeout=10)
        print(f"✓ Basic connection successful! Status: {response.status_code}")
        print(f"Response: {response.text[:100]}...")
        return True
    except Exception as e:
        print(f"✗ Basic connection failed: {e}")
        return False

def test_capture():
    """Test capture endpoint"""
    print("\nTesting capture endpoint...")
    try:
        response = requests.get(f"http://{ESP32_IP}/capture", timeout=10)
        if response.status_code == 200:
            print("✓ Capture endpoint working!")
            
            # Save captured image
            with open("test_capture.jpg", "wb") as f:
                f.write(response.content)
            print("✓ Image saved as 'test_capture.jpg'")
            
            # Try to display with OpenCV
            nparr = np.frombuffer(response.content, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if img is not None:
                print("✓ Image decoded successfully!")
                cv2.imshow("ESP32-CAM Capture Test", img)
                print("Press any key to close image window...")
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                print("✗ Failed to decode image")
            return True
        else:
            print(f"✗ Capture failed. Status: {response.status_code}")
            return False
    except Exception as e:
        print(f"✗ Capture test failed: {e}")
        return False

def test_stream_opencv():
    """Test stream dengan OpenCV - Method 1"""
    print("Method 1: Testing with OpenCV...")
    try:
        # Coba beberapa konfigurasi OpenCV
        cap = cv2.VideoCapture(f"http://{ESP32_IP}/stream")
        
        # Set buffer size minimal untuk mengurangi latency
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        if cap.isOpened():
            print("✓ OpenCV stream opened successfully!")
            
            # Coba baca beberapa frame
            for attempt in range(5):
                ret, frame = cap.read()
                if ret and frame is not None:
                    print(f"✓ OpenCV frame {attempt + 1} read successfully!")
                    if attempt == 0:  # Show first frame
                        cv2.imshow("OpenCV Stream Test", frame)
                        print("Press any key to close OpenCV test...")
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()
                    break
                else:
                    print(f"✗ Attempt {attempt + 1}: Cannot read frame")
                    time.sleep(0.1)
            
            cap.release()
            return ret
        else:
            print("✗ OpenCV cannot open stream")
            return False
    except Exception as e:
        print(f"✗ OpenCV method failed: {e}")
        return False

def parse_mjpeg_stream(stream_url, max_frames=30):
    """Parse MJPEG stream dengan boundary detection yang lebih robust"""
    try:
        import urllib.request
        
        req = urllib.request.Request(stream_url)
        response = urllib.request.urlopen(req, timeout=15)
        
        print("✓ urllib opened stream successfully!")
        print(f"Content-Type: {response.headers.get('content-type')}")
        
        # Buffer untuk mengumpulkan data
        buffer = b''
        frame_count = 0
        start_time = time.time()
        
        print("Reading stream data... Press Ctrl+C to stop")
        
        while frame_count < max_frames:
            # Baca chunk data
            chunk = response.read(4096)  # Increased chunk size
            if not chunk:
                print("No more data from stream")
                break
            
            buffer += chunk
            
            # Cari marker JPEG
            while True:
                # Cari start of JPEG (0xFFD8)
                jpeg_start = buffer.find(b'\xff\xd8')
                if jpeg_start == -1:
                    break
                
                # Cari end of JPEG (0xFFD9) setelah start
                jpeg_end = buffer.find(b'\xff\xd9', jpeg_start)
                if jpeg_end == -1:
                    break
                
                # Extract JPEG frame
                jpeg_data = buffer[jpeg_start:jpeg_end + 2]
                buffer = buffer[jpeg_end + 2:]
                
                # Validasi ukuran minimum untuk JPEG
                if len(jpeg_data) < 100:  # JPEG terlalu kecil
                    continue
                
                try:
                    # Decode frame
                    nparr = np.frombuffer(jpeg_data, np.uint8)
                    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                    
                    if frame is not None and frame.size > 0:
                        frame_count += 1
                        
                        if frame_count == 1:
                            print("✓ Successfully decoded first frame!")
                            print(f"Frame size: {frame.shape}")
                            cv2.imshow("urllib Stream Test", frame)
                            print("Press any key to continue test...")
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
                        
                        if frame_count % 10 == 0:
                            elapsed = time.time() - start_time
                            fps = frame_count / elapsed if elapsed > 0 else 0
                            print(f"Frames received: {frame_count}, FPS: {fps:.2f}")
                    
                except Exception as decode_error:
                    print(f"Frame decode error: {decode_error}")
                    continue
        
        response.close()
        
        if frame_count > 0:
            elapsed = time.time() - start_time
            avg_fps = frame_count / elapsed if elapsed > 0 else 0
            print(f"✓ urllib method successful! Received {frame_count} frames, Avg FPS: {avg_fps:.2f}")
            return True
        else:
            print("✗ No frames were successfully decoded")
            return False
            
    except KeyboardInterrupt:
        print("\n✓ urllib test stopped by user")
        return True
    except Exception as e:
        print(f"✗ urllib method failed: {e}")
        return False

def test_stream_requests():
    """Test stream dengan requests - Method 3"""
    print("\nMethod 3: Testing with requests streaming...")
    try:
        # Gunakan session untuk koneksi yang lebih stabil
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        
        response = session.get(f"http://{ESP32_IP}/stream", 
                             stream=True, 
                             timeout=15)
        
        if response.status_code == 200:
            print("✓ requests stream opened successfully!")
            print(f"Content-Type: {response.headers.get('content-type')}")
            
            buffer = b''
            frame_count = 0
            start_time = time.time()
            
            for chunk in response.iter_content(chunk_size=4096):
                if frame_count >= 10:  # Test 10 frames
                    break
                
                if not chunk:
                    continue
                
                buffer += chunk
                
                # Parse JPEG frames
                while True:
                    jpeg_start = buffer.find(b'\xff\xd8')
                    if jpeg_start == -1:
                        break
                    
                    jpeg_end = buffer.find(b'\xff\xd9', jpeg_start)
                    if jpeg_end == -1:
                        break
                    
                    jpeg_data = buffer[jpeg_start:jpeg_end + 2]
                    buffer = buffer[jpeg_end + 2:]
                    
                    if len(jpeg_data) < 100:
                        continue
                    
                    try:
                        nparr = np.frombuffer(jpeg_data, np.uint8)
                        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        
                        if frame is not None and frame.size > 0:
                            frame_count += 1
                            print(f"✓ Frame {frame_count} decoded successfully")
                            
                            if frame_count == 1:
                                cv2.imshow("requests Stream Test", frame)
                                print("Press any key to continue...")
                                cv2.waitKey(0)
                                cv2.destroyAllWindows()
                    
                    except Exception as decode_error:
                        continue
            
            session.close()
            
            if frame_count > 0:
                elapsed = time.time() - start_time
                avg_fps = frame_count / elapsed if elapsed > 0 else 0
                print(f"✓ requests method successful! Received {frame_count} frames, Avg FPS: {avg_fps:.2f}")
                return True
            else:
                print("✗ No frames decoded with requests method")
                return False
        else:
            print(f"✗ requests stream failed with status: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"✗ requests method failed: {e}")
        return False

def test_stream():
    """Test stream endpoint dengan multiple methods"""
    print("\nTesting stream endpoint...")
    
    success_count = 0
    
    # Method 1: OpenCV
    if test_stream_opencv():
        success_count += 1
    
    # Method 2: urllib (MJPEG parsing)
    print("\nMethod 2: Testing with urllib (MJPEG parsing)...")
    if parse_mjpeg_stream(f"http://{ESP32_IP}/stream"):
        success_count += 1
    
    # Method 3: requests
    if test_stream_requests():
        success_count += 1
    
    if success_count > 0:
        print(f"\n✓ Streaming working! {success_count}/3 methods successful")
        return True
    else:
        print(f"\n✗ All streaming methods failed")
        return False

def main():
    print("ESP32-CAM Connection Test (Enhanced)")
    print("=" * 40)
    print(f"ESP32-CAM IP: {ESP32_IP}")
    print()
    
    # Test basic connection
    if not test_basic_connection():
        print("\n❌ Basic connection failed. Check:")
        print("1. ESP32-CAM is powered on")
        print("2. WiFi connection is stable") 
        print("3. Computer and ESP32-CAM are on same network")
        print("4. Firewall is not blocking the connection")
        return
    
    # Test capture
    if not test_capture():
        print("\n❌ Capture test failed")
        return
    
    # Test stream
    test_stream()
    
    print("\n✅ All tests completed!")
    print(f"\nYour ESP32-CAM URLs:")
    print(f"- Main page: http://{ESP32_IP}/")
    print(f"- Stream: http://{ESP32_IP}/stream")
    print(f"- Capture: http://{ESP32_IP}/capture")

if __name__ == "__main__":
    main()